In [ ]:
pip install fastquant

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import backtrader as bt

In [ ]:
from fastquant import backtest,get_crypto_data
import numpy as np

In [ ]:
df = get_crypto_data("BTCUSDT",start_date='2021-01-01',end_date='2021-12-27', time_resolution='1h')

In [ ]:
df

,open,high,low,close,volume
dt,,,,,
2020-12-31 19:00:00,28872.24,29000.00,28742.41,28897.83,2293.821339
2020-12-31 20:00:00,28897.84,29139.65,28862.00,29126.70,1936.480299
2020-12-31 21:00:00,29126.71,29169.55,28900.79,28966.36,2524.473111
2020-12-31 22:00:00,28966.36,29143.73,28910.19,29100.84,1438.506326
2020-12-31 23:00:00,29100.83,29110.35,28780.00,28923.63,1976.419299
...,...,...,...,...,...
2021-12-26 20:00:00,50342.51,50382.61,50265.97,50269.44,387.961600
2021-12-26 21:00:00,50269.44,51040.00,49914.73,50774.16,1890.386080
2021-12-26 22:00:00,50776.16,51280.00,50772.00,50973.64,1221.970650


In [ ]:
from scipy.ndimage import convolve1d as conv
df['MA'] = df['close'].rolling(window=14).mean()
def bb_indicator(data):
  
  df['STD'] = df['close'].rolling(window=14).std()

  df['Upper_Band'] = df['MA'] + (df['STD'] * 2)

  df['Lower_Band'] = df['MA'] - (df['STD'] * 2)

In [ ]:
bb_indicator(df)

In [ ]:
df.tail()

,open,high,low,close,volume,MA,STD,Upper_Band,Lower_Band
dt,,,,,,,,,
2021-12-26 20:00:00,50342.51,50382.61,50265.97,50269.44,387.96160,50055.430000,202.935386,50461.300771,49649.559229
2021-12-26 21:00:00,50269.44,51040.00,49914.73,50774.16,1890.38608,50115.699286,275.326438,50666.352161,49565.046410
2021-12-26 22:00:00,50776.16,51280.00,50772.00,50973.64,1221.97065,50193.675000,348.819439,50891.313878,49496.036122
2021-12-26 23:00:00,50973.65,51062.04,50638.16,50775.49,656.46684,50248.167143,376.821121,51001.809384,49494.524902
2021-12-27 00:00:00,50775.48,50974.97,50567.60,50622.87,1018.69342,50286.944286,386.004323,51058.952931,49514.935640


In [ ]:
import matplotlib.pyplot as pl
pl.style.use("default")

In [ ]:
import backtrader as bt
import backtrader.analyzers as btanalyzers

In [ ]:
from fastquant import CustomStrategy, BaseStrategy
from fastquant.indicators import MACD, CrossOver 
from fastquant.indicators.custom import CustomIndicator


# Create a subclass of the BaseStrategy, We call this MAMAStrategy (MACD + ALMA)
class MAMAStrategy(BaseStrategy):
    
    count = 0
    
    params = (
        ("upper_band", "Upper_Band"),   # name for the ALMA column from the dataframe
        ("lower_band", "Lower_Band"),  
        ("rsi_period", 14),  
        ("rsi_upper", 70),
        ("rsi_lower",30)
    )

    def __init__(self):
        # Initialize global variables
        super().__init__()
        
        self.rsi_period = self.params.rsi_period
        self.rsi_upper = self.params.rsi_upper
        self.rsi_lower = self.params.rsi_lower
        self.rsi = bt.indicators.RelativeStrengthIndex(period=self.rsi_period, upperband=self.rsi_upper, lowerband=self.rsi_lower)

    

        # Assign ALMA column from the dataframe
        self.upper_band = self.params.upper_band
        self.lower_band = self.params.lower_band
        
        # Set ALMA indicator from the alma column of data
        self.Upper_Band = CustomIndicator(
            self.data, custom_column=self.upper_band,
        )
        self.Lower_Band = CustomIndicator(
            self.data, custom_column=self.lower_band,
        )
        

        

    # Buy when the custom indicator is below the lower limit, and sell when it's above the upper limit
    

    def buy_signal(self):
        
        bb_buy =  self.data.close[0] < self.Lower_Band    # Close is above ALMA
        rsi_buy = self.rsi < self.rsi_lower# MACD crosses signal line upward
        
        return rsi_buy and bb_buy
    
    def sell_signal(self):
        
        
        bb_sell = self.Upper_Band < self.data.close[0]
        rsi_sell = self.rsi > self.rsi_upper

        return rsi_sell and bb_sell 



In [ ]:
results,history = backtest(MAMAStrategy, df,  init_cash=100000, verbose=True, return_history=True, plot=False)

Starting Portfolio Value: 100000.00
2021-12-27T00:00:00, ===Global level arguments===
2021-12-27T00:00:00, init_cash : 100000
2021-12-27T00:00:00, buy_prop : 1
2021-12-27T00:00:00, sell_prop : 1
2021-12-27T00:00:00, commission : 0
2021-12-27T00:00:00, stop_loss : 0
2021-12-27T00:00:00, stop_trail : 0
2021-12-27T00:00:00, take_profit : 0
2021-12-27T00:00:00, Final Portfolio Value: 104417.47999999995
2021-12-27T00:00:00, Final PnL: 4417.48
Time used (seconds): 6.555549144744873
Number of strat runs: 1
Number of strats per run: 1
Strat names: ['MAMAStrategy']
**************************************************
--------------------------------------------------
Strategy Parameters	init_cash:100000	buy_prop:1	sell_prop:1	fractional:False	commission:0	stop_loss:0	stop_trail:0	take_profit:0	execution_type:close	channel:	symbol:	allow_short:False	short_max:1.5	add_cash_amount:0	add_cash_freq:M	invest_div:True	upper_band:Upper_Band	lower_band:Lower_Band	rsi_period:14	rsi_upper:70	rsi_lower:30
Re

In [ ]:
results

,strat_id,init_cash,buy_prop,sell_prop,fractional,commission,stop_loss,stop_trail,take_profit,execution_type,...,won,lost,won_avg,won_avg_prcnt,lost_avg,lost_avg_prcnt,won_max,won_max_prcnt,lost_max,lost_max_prcnt
0,0,100000,1,1,False,0,0,0,0,close,...,13,10,6017.712308,6.017712,-7570.74,-7.57074,19495.8,19.4958,-32990.84,-32.99084


In [ ]:
results.columns

Index(['strat_id', 'init_cash', 'buy_prop', 'sell_prop', 'fractional',
       'commission', 'stop_loss', 'stop_trail', 'take_profit',
       'execution_type', 'channel', 'symbol', 'allow_short', 'short_max',
       'add_cash_amount', 'add_cash_freq', 'invest_div', 'upper_band',
       'lower_band', 'rsi_period', 'rsi_upper', 'rsi_lower', 'rtot', 'ravg',
       'rnorm', 'rnorm100', 'len', 'drawdown', 'moneydown', 'max',
       'maxdrawdown', 'maxdrawdownperiod', 'sharperatio', 'pnl', 'final_value',
       'total', 'win_rate', 'won', 'lost', 'won_avg', 'won_avg_prcnt',
       'lost_avg', 'lost_avg_prcnt', 'won_max', 'won_max_prcnt', 'lost_max',
       'lost_max_prcnt'],
      dtype='object')

In [ ]:
results[['rsi_upper', 'rsi_lower', 'maxdrawdown', 'maxdrawdownperiod', 'sharperatio', 'pnl','win_rate' ]]

,rsi_upper,rsi_lower,maxdrawdown,maxdrawdownperiod,sharperatio,pnl,win_rate
0,70,30,34.234104,231,0.547253,4417.48,0.541667


In [ ]:
history['orders']

,strat_id,strat_name,dt,type,price,size,value,commission,pnl
0,0,,2021-01-04 11:00:00,buy,30270.76,3,90812.28,0.0,0.00
1,0,,2021-01-07 00:00:00,sell,36769.36,-3,90812.28,0.0,19495.80
2,0,,2021-01-10 20:00:00,buy,37456.77,3,112370.31,0.0,0.00
3,0,,2021-01-15 00:00:00,sell,39144.50,-3,112370.31,0.0,5063.19
4,0,,2021-01-15 16:00:00,buy,35610.72,3,106832.16,0.0,0.00
5,0,,2021-01-26 00:00:00,sell,32254.20,-3,106832.16,0.0,-10069.56
6,0,,2021-01-27 15:00:00,buy,29573.15,3,88719.45,0.0,0.00
7,0,,2021-01-30 00:00:00,sell,34252.20,-3,88719.45,0.0,14037.15
8,0,,2021-02-22 14:00:00,buy,52070.47,2,104140.94,0.0,0.00
9,0,,2021-03-02 00:00:00,sell,49587.03,-2,104140.94,0.0,-4966.88


In [ ]:
history['indicators']

,strat_id,strat_name,dt,RelativeStrengthIndex_14,CustomIndicator
0,0,,2020-12-31 19:00:00,NaN,NaN
1,0,,2020-12-31 20:00:00,NaN,NaN
2,0,,2020-12-31 21:00:00,NaN,NaN
3,0,,2020-12-31 22:00:00,NaN,NaN
4,0,,2020-12-31 23:00:00,NaN,NaN
...,...,...,...,...,...
8628,0,,2021-12-26 20:00:00,51.367807,49649.559229
8629,0,,2021-12-26 21:00:00,63.833765,49565.046410
8630,0,,2021-12-26 22:00:00,67.391445,49496.036122
8631,0,,2021-12-26 23:00:00,60.974996,49494.524902


In [ ]:
history

{'orders':     strat_id strat_name                  dt  type     price  size      value  \
 0          0            2021-01-04 11:00:00   buy  30270.76     3   90812.28   
 1          0            2021-01-07 00:00:00  sell  36769.36    -3   90812.28   
 2          0            2021-01-10 20:00:00   buy  37456.77     3  112370.31   
 3          0            2021-01-15 00:00:00  sell  39144.50    -3  112370.31   
 4          0            2021-01-15 16:00:00   buy  35610.72     3  106832.16   
 5          0            2021-01-26 00:00:00  sell  32254.20    -3  106832.16   
 6          0            2021-01-27 15:00:00   buy  29573.15     3   88719.45   
 7          0            2021-01-30 00:00:00  sell  34252.20    -3   88719.45   
 8          0            2021-02-22 14:00:00   buy  52070.47     2  104140.94   
 9          0            2021-03-02 00:00:00  sell  49587.03    -2  104140.94   
 10         0            2021-03-15 10:00:00   buy  55867.91     2  111735.82   
 11         0     